In [1815]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
import random
import time
import math

In [989]:
racepage_urls = []

In [1024]:
url = "https://raceraves.com/find-a-race/?find_a_race_form=849414127a&_wp_http_referer=%2Ffind-a-race%2F&keyword&radius=5&from-date=Dec%201,%202017&to-date=Dec%2031,%202017&paged=7"
UAS = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1", 
       "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
       "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
       "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
       )

ua = UAS[random.randrange(len(UAS))]

headers = {'user-agent': ua}
response = requests.get(url, headers=headers)

In [1025]:
response.status_code

200

In [1026]:
race_pages = BeautifulSoup(response.content, 'lxml')

In [1027]:
divs = race_pages.find_all('div')
divdivs = divs[0].find_all('div')#, {"class": "to-date-wrapper"})
atags = divdivs[8].find_all('a')
for atag in atags:
    if atag.attrs['href'][0:28] == 'https://raceraves.com/races/':
        racepage_urls.append(atag.attrs['href'])
racepage_urls

['https://raceraves.com/races/run-the-lights-5k/',
 'https://raceraves.com/races/mistletoe-run/',
 'https://raceraves.com/races/poop-trail-run/',
 'https://raceraves.com/races/st-helens-reindeer-run/',
 'https://raceraves.com/races/reindeer-run-charleston-sc/',
 'https://raceraves.com/races/devil-dog-ultras/',
 'https://raceraves.com/races/perpetual-motion/',
 'https://raceraves.com/races/redbird-crest-100k/',
 'https://raceraves.com/races/lovit-trail-marathon/',
 'https://raceraves.com/races/tejas-trails-mosaic-trail-run/',
 'https://raceraves.com/races/hot-cider-hustle-indianapolis-in/',
 'https://raceraves.com/races/peacock-gap/',
 'https://raceraves.com/races/heroes-in-recovery-6k-austin/',
 'https://raceraves.com/races/the-phoenician-half-marathon/',
 'https://raceraves.com/races/sea-island-half-marathon/',
 'https://raceraves.com/races/race-13-1-baltimore-md/',
 'https://raceraves.com/races/lesotho-high-altitude-summer-marathon/',
 'https://raceraves.com/races/great-pacific-marat

In [1028]:
racepage_df = pd.DataFrame(racepage_urls, columns=['RacePage_URL'])
print(len(racepage_df))
racepage_df.to_csv('RacePages_Dec2017.csv')

331


In [2049]:
racepage_urls = pd.read_csv('RacePages_Nov2017.csv')
racepage_urls = racepage_urls.drop("Unnamed: 0", axis=1)
# racepage_urls = racepage_urls.head(3)
racepage_urls.head(10)

,RacePage_URL
0,https://raceraves.com/races/swamp-fox-ultra/
1,https://raceraves.com/races/save-the-daylight/
2,https://raceraves.com/races/ragnar-trail-relay...
3,https://raceraves.com/races/tbunk-endurance-ch...
4,https://raceraves.com/races/ragnar-relay-napa-...
5,https://raceraves.com/races/disney-wine-dine-h...
6,https://raceraves.com/races/kaiser-permanente-...
7,https://raceraves.com/races/in-unity-we-run/
8,https://raceraves.com/races/the-hamsterwheel/
9,https://raceraves.com/races/run-for-a-safe-haven/


In [2050]:
all_races = {}
reviewer_ids = []
# all_reviews = {}

In [2051]:
# racepage_urls['RacePage_URL'].iloc[57:]

In [2052]:
for url in racepage_urls['RacePage_URL']:
#     UAS = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1", 
#            "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
#            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
#            "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
#            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
#            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
#            )

    ua = "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0"

    headers = {'user-agent': ua}
    status = 999
    while status != 200:
        response = requests.get(url, headers=headers)
        status = response.status_code
    race_page = BeautifulSoup(response.content, 'lxml')

    #Race Details
    if race_page.find('h1', {'class': "entry-title"}) != None:
        race_details = {}
        race_details['name'] = race_page.find('h1', {'class': "entry-title"}).contents[0]
        race_location_raw = str(race_page.find('h2', {'class': "entry-subtitle"}).contents[1].contents)
        location_end = race_location_raw.find("\\t\\t\'")
        race_details['location'] = race_location_raw[8: location_end]
        race_details['state'] = race_details['location'][(race_details['location'].find(',') + 2):]
        if len(race_details['state']) > 2:
            race_details['region'] = 'Non-US'
        else:
            race_details['region'] = 'US'

        lat_long = race_page.find('div', {'class': "race-section--map__container"})
        race_details['latitude'] = lat_long.attrs['data-wds-rr-map-lat']
        race_details['longitude'] = lat_long.attrs['data-wds-rr-map-lng']

        race_date_raw = str(race_page.find('h2').find('span', {'class': "mobile-race-date"}).contents)
        date_end = race_date_raw.find(',') + 6
        race_details['date'] = race_date_raw[8: date_end]

        race_table = race_page.find('div', {'class': "race-section__content--left"})
        race_items = race_table.find_all('li', {'class': "list-stripped__item"})
        if len(race_items) == 3:
            race_details['distances'] = race_items[0].contents
            race_details['course type'] = race_items[1].contents[0]
            race_details['website'] = race_items[2].find('a').attrs['href']
        else:
            race_details['distances'] = race_items[0].contents
            race_details['website'] = race_items[1].find('a').attrs['href']        
        race_desc_raw = race_page.find('div', {'class': "full-description"})
        if race_desc_raw == []:
            race_desc_raw = race_page.find('div', {'class': "compact-description"})
        try:
            race_description = ''
            for paragraph in race_desc_raw.find_all('p'):
                try:
                    race_description = race_description + ' ' + paragraph.contents[0]
                except:
                    race_description = race_description + ' ' + paragraph.contents[1]
        except:
            race_description = ''
        race_details['description'] = race_description


        #Race Weather
        weather_table = race_page.find('table', {'class': "race-section--weather__content"})
        weather_elements = weather_table.find_all('td')

        race_details['weather_scale'] = weather_elements[12].contents[0][-2]

        race_details['weather_year5'] = {
            'year': weather_elements[1].contents[0],
            'conditions': weather_elements[7].find('canvas').attrs['alt'],
            'high': int(weather_elements[13].contents[0]),
            'low': int(weather_elements[19].contents[0])
        }

        race_details['weather_year4'] = {
            'year': weather_elements[2].contents[0],
            'conditions': weather_elements[8].find('canvas').attrs['alt'],
            'high': int(weather_elements[14].contents[0]),
            'low': int(weather_elements[20].contents[0])
        }

        race_details['weather_year3'] = {
            'year': weather_elements[3].contents[0],
            'conditions': weather_elements[9].find('canvas').attrs['alt'],
            'high': int(weather_elements[15].contents[0]),
            'low': int(weather_elements[21].contents[0])
        }

        race_details['weather_year2'] = {
            'year': weather_elements[4].contents[0],
            'conditions': weather_elements[10].find('canvas').attrs['alt'],
            'high': int(weather_elements[16].contents[0]),
            'low': int(weather_elements[22].contents[0])
        }

        race_details['weather_year1'] = {
            'year': weather_elements[5].contents[0],
            'conditions': weather_elements[11].find('canvas').attrs['alt'],
            'high': int(weather_elements[17].contents[0]),
            'low': int(weather_elements[23].contents[0])
        }


        #Ratings
        race_details['overall_rating'] = float(race_page.find('span', {"itemprop": "ratingValue"}).contents[0])
        ratings = race_page.find_all('div', {'class': "race-section--rating__ratings__rate"})
        race_details['diff_rating'] = float(ratings[0].contents[0])
        race_details['scenery_rating'] = float(ratings[1].contents[0])
        race_details['prod_rating'] = float(ratings[2].contents[0])
        race_details['swag_rating'] = float(ratings[3].contents[0])

        all_races[race_details['name']] = race_details


        reviewers = race_page.find_all('div', {"class": "race-comment__content"})
        for i in range(int(len(reviewers)/2)):
            content = reviewers[i].find('div', {"class": "comment-author__content"})
            reviewer_ids.append(content.find('a', {"class": "comment-author__text"}).contents[1].contents[0])
        
    time.sleep(15)

In [2053]:
races_df = pd.DataFrame.from_dict(all_races, orient='index')
races_df.to_csv('Races_Nov2017.csv')

reviewers_df = pd.DataFrame(reviewer_ids, columns=['Reviewer_ID'])
reviewers_df.to_csv('Reviewers_Nov2017.csv')

In [2045]:
races_df.head()

,name,location,state,region,latitude,longitude,date,distances,course type,website,...,weather_year5,weather_year4,weather_year3,weather_year2,weather_year1,overall_rating,diff_rating,scenery_rating,prod_rating,swag_rating
"10K at 10,000 Feet","10K at 10,000 Feet","Jackson County, CO",CO,US,40.4155932,-106.6208697,"Sep 3, 2017","[10K, 5K]",Trail (Unpaved),http://www.runningseries.com/races/10k_10000ft...,...,"{'year': 2017, 'conditions': 'partly-cloudy-ni...","{'year': 2016, 'conditions': 'rain', 'high': 7...","{'year': 2015, 'conditions': 'rain', 'high': 6...","{'year': 2014, 'conditions': 'clear-day', 'hig...","{'year': 2013, 'conditions': 'clear-day', 'hig...",0.0,0.0,0.0,0.0,0.0
12-Hour Adventure Trail Run,12-Hour Adventure Trail Run,"Triangle, VA",VA,US,38.5598822,-77.3686316,"Sep 16, 2017","[12 hr, 12 hr Relay]",Trail (Unpaved),http://www.athletic-equation.com/12-HR_ATR.html,...,"{'year': 2017, 'conditions': 'partly-cloudy-da...","{'year': 2016, 'conditions': 'partly-cloudy-ni...","{'year': 2015, 'conditions': 'clear-day', 'hig...","{'year': 2014, 'conditions': 'partly-cloudy-da...","{'year': 2013, 'conditions': 'rain', 'high': 7...",0.0,0.0,0.0,0.0,0.0
5 Peaks Trail Running Series: Heart Lake,5 Peaks Trail Running Series: Heart Lake,"Brampton, Canada",Canada,Non-US,43.7424456,-79.7958169,"Sep 16, 2017","[11K, 7.5K, 3K, 1K]",Trail (Unpaved),http://www.5peaks.com/shop/heart-lake-race-4,...,"{'year': 2017, 'conditions': 'clear-day', 'hig...","{'year': 2016, 'conditions': 'clear-day', 'hig...","{'year': 2015, 'conditions': 'clear-day', 'hig...","{'year': 2014, 'conditions': 'partly-cloudy-da...","{'year': 2013, 'conditions': 'clear-day', 'hig...",0.0,0.0,0.0,0.0,0.0
5K Family Run/Walk to STOMP Out Epilepsy and SUDEP,5K Family Run/Walk to STOMP Out Epilepsy and S...,"Wilmette, IL",IL,US,42.0843015,-87.6972713,"Sep 17, 2017",[5K],Road (Paved),https://go.evvnt.com/148124-0,...,"{'year': 2017, 'conditions': 'rain', 'high': 7...","{'year': 2016, 'conditions': 'partly-cloudy-da...","{'year': 2015, 'conditions': 'rain', 'high': 8...","{'year': 2014, 'conditions': 'partly-cloudy-da...","{'year': 2013, 'conditions': 'clear-day', 'hig...",0.0,0.0,0.0,0.0,0.0
5K Run for Everyone,5K Run for Everyone,"Westfield, NJ",NJ,US,40.650552,-74.3524134,"Sep 10, 2017",[5K],Road (Paved),https://www.westfieldynj.org/get-involved/5k-r...,...,"{'year': 2017, 'conditions': 'clear-day', 'hig...","{'year': 2016, 'conditions': 'partly-cloudy-da...","{'year': 2015, 'conditions': 'rain', 'high': 7...","{'year': 2014, 'conditions': 'partly-cloudy-ni...","{'year': 2013, 'conditions': 'clear-day', 'hig...",0.0,0.0,0.0,0.0,0.0


In [2046]:
reviewers_df.head()

,Reviewer_ID
0,dana.0212
1,faerybard
2,KelliKO
3,portlandrunner
4,dtm3000


In [2047]:
races_df.shape

(736, 22)

In [2048]:
reviewers_df.shape

(509, 1)

In [1984]:
url

'https://raceraves.com/races/406-duathlon-challenge-5k/'

In [1982]:
racepage_urls[racepage_urls['RacePage_URL'] == url]

,RacePage_URL
297,https://raceraves.com/races/406-duathlon-chall...


In [2148]:
users = pd.read_csv('./RaceRaves_data/Unique_RacerIDs.csv', header=None, names=['racer_id'])
users.head()

,racer_id
0,808runwarrior
1,Aherrell
2,Bazhinaa
3,COLazyTurtle
4,Chew17


In [2152]:
# unique_users = list(set(users['Reviewer_ID']))
# unique_users = [user.replace('.', '-') for user in unique_users]
# unique_users = [user.strip('@') for user in unique_users]
# len(unique_users)

In [2151]:
unique_users = list(users['racer_id'])
len(unique_users)

2010

In [2150]:
unique_users[0:10]

['808runwarrior',
 'Aherrell',
 'Bazhinaa',
 'COLazyTurtle',
 'Chew17',
 'ChrisCruz671',
 'DBell',
 'Daleannon',
 'Donna',
 'HrmeetChpra']

In [2169]:
racers = {}

In [2180]:
for user in unique_users[1563:]:
    url = "https://raceraves.com/" + user + "/"
#     UAS = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1", 
#            "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
#            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
#            "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
#            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
#            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
#            )

    ua = "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36"

    headers = {'user-agent': ua}
    status = 999
    while status != 200:
        response = requests.get(url, headers=headers)
        status = response.status_code
        if status == 404:
            break
    racer_page = BeautifulSoup(response.content, 'lxml')
    
    racer = {}
    racer['racer_id'] = url[22:-1]
    try:
        racer['location'] = \
        racer_page.find('span', {'class': "location-message bp-profile__info__field"}).contents[0]
        racer['state'] = racer['location'][(racer['location'].find(',') + 2):]
        if len(racer['state']) == 2:
            racer['region'] = 'US'
        else:
            racer['region'] = 'Non-US'
    except:
        race['location'] = ''
        racer['state'] = ''
        racer['region'] = ''
    racer['since_date'] = \
        racer_page.find('span', {'class': "raving-since bp-profile__info__field"}).contents[0]
    try:
        racer['affiliations'] = \
            racer_page.find('span', {'class': "affiliations bp-profile__info__field"}).contents[0].split(', ')
    except:
        racer['affiliations'] = ''
    racer['last active'] = racer_page.find('span', {'class': "activity"}).contents[0]

    try:
        about_me = racer_page.find('div', {'class': "profile-fields"})
        about_me_items = about_me.find_all('span', {'class': "data"})
        try:
            racer['clubs'] = about_me_items[0].contents[1].contents[0].split(', ')
        except:
            racer['clubs'] = []
        try:
            racer['rave_race'] = about_me_items[1].contents[1].contents[0]
        except:
            racer['rave_race'] = ''
        try:
            racer['dream_race'] = about_me_items[2].contents[1].contents[0]
        except:
            racer['dream_race'] = ''
        try:
            racer['run_because'] = about_me_items[3].contents[1].contents[0]
        except:
            racer['run_because'] = ''
    except:
        racer['clubs'] = ''
        racer['rave_race'] = ''
        racer['dream_race'] = ''
        racer['run_because'] = ''

    past_race_table = racer_page.find('table', {'id': "past-races-table"})
    racer['past_races'] = {}
    for i in range(1, len(past_race_table.find_all('tr'))):
        past_race = {}
        past_race_raw = past_race_table.find_all('tr')[i]
        past_race['name'] = past_race_raw.find('a').contents[0].strip('\n').strip('\t')
        past_race['distance'] = past_race_raw.find('td', {'class': "race-distance"}).contents[0].strip('\n').strip('\t')
        past_race['location'] = past_race_raw.find('td', {'class': "race-location"}).contents[0].strip('\n').strip('\t')
        past_race['date'] = past_race_raw.find('td', {'class': "race-date"}).contents[0].strip('\n').strip('\t')
        past_race['result'] = past_race_raw.find('td', {'class': "race-result"}).contents[0].strip('\n').strip('\t')
        racer['past_races'][past_race['name']] = past_race

    racer_reviews = {}
    reviews = racer_page.find_all('div', {'class': "byuser"})
    overall_raw = racer_page.find_all('div', {'class': 'race-comment__score--mobile'})
    for i in range(len(reviews)):
        race_name = reviews[i].find('a').contents[0]
        racer_reviews[race_name] = []
    for i in range(len(reviews)):
        review = {}
        review['race_name'] = reviews[i].find('a').contents[0]
        review['title'] = reviews[i].find('h4', {'class': "race-comment__title"}).contents[0]
        details = reviews[i].find('a', {'class': "race-comment__subtitle"})
        details = details.find('span').contents[0].split('|')
        review['date'] = details[0].strip('\n').strip('\t')[:-1]
        review['distance'] = details[1].strip('\n').strip('\t')
        review['type'] = details[2].strip('\n').strip('\t')
        review_text = reviews[i].find('article', {'class': "race-comment__comment--full"})
        review['text'] = ''
        for paragraph in review_text.find_all('p'):
            try:
                review['text'] = review['text'] + ' ' + paragraph.contents[0]
            except:
                continue

        rating_string = str(overall_raw[i])[(str(overall_raw[i]).find("width")-12):][0:3].strip('-')
        review['overall_rating'] = float(rating_string) / 20
        ratings_raw = reviews[i].find_all('div', {"class": "race-comment__rating-count"})
        review['diff_rating'] = float(ratings_raw[0].contents[0])
        review['prod_rating'] = float(ratings_raw[1].contents[0])
        review['scenery_rating'] = float(ratings_raw[2].contents[0])
        review['swag_rating'] = float(ratings_raw[3].contents[0])
        racer_reviews[review['race_name']].append(review)

    racer['reviews'] = racer_reviews

    racers[racer['racer_id']] = racer
    time.sleep(15)

In [2181]:
racers_df = pd.DataFrame.from_dict(racers, orient='index')
racers_df.to_csv('Racers_All.csv')
racers_df.head()

,racer_id,location,state,region,since_date,affiliations,last active,clubs,rave_race,dream_race,run_because,past_races,reviews
090775candice,090775candice,"Anaheim, CA",CA,US,Raving since 2017,,active 1 year ago,[],,Doing a spartan race.,I want to show people if you believe in yourse...,{'Tustin Hangar Half Marathon & 5K': {'name': ...,{Tustin Hangar Half Marathon & 5K: [{'race_nam...
16490derek,16490derek,"Wilmot, Canada",Canada,Non-US,Raving since 2017,,"active 4 months, 1 week ago",[],,,,{'Hamilton Marathon Road2Hope': {'name': 'Hami...,{Hamilton Marathon Road2Hope: [{'race_name': H...
18anjel,18anjel,"Chino, CA",CA,US,Raving since 2016,[Half Fanatic],"active 2 years, 1 month ago",[],,OC Marathon and Half,My family runs. We run as a family hobby.,{'Surf City Marathon & Half Marathon': {'name'...,{Surf City Marathon & Half Marathon: [{'race_n...
1FlyMother,1FlyMother,"Port Allegany, PA",PA,US,Raving since 2017,,active 8 months ago,[],,,,{'Presque Isle Half Marathon': {'name': 'Presq...,{Presque Isle Half Marathon: [{'race_name': Pr...
1ladybugangel,1ladybugangel,"Oxnard, CA",CA,US,Raving since 2017,,"active 4 months, 3 weeks ago",[Twisted Trail Runners and Twisted Trail runn...,Spacerock 2017,"Valencia Trail Race March 31st, 2018",It makes me feel like I am alive and living in...,{'SPACEROCK Trail Race': {'name': 'SPACEROCK T...,{SPACEROCK Trail Race: [{'race_name': SPACEROC...


In [2182]:
racers_df.shape

(2010, 13)

In [2183]:
user

'zlyvlk1'

In [2184]:
unique_users.index(user)

2009